In [8]:
import utils
import skimage
import skimage.morphology
import numpy as np

In [9]:
# Remove noise function from subtask 3a)
def remove_noise(im: np.ndarray) -> np.ndarray:
    """
        A function that removes noise in the input image.
        args:
            im: np.ndarray of shape (H, W) with boolean values (dtype=np.bool)
        return:
            (np.ndarray) of shape (H, W). dtype=np.bool
    """
    ### START YOUR CODE HERE ### (You can change anything inside this block)
    # We choose an adequate structuring element size to eliminate all the specks of noise in original image
    struct_size = 15 # Should be an odd, positive integer
    structuring_element = np.zeros([struct_size, struct_size])
    # We construct a binary circle centered in (floor(struct_radius/2), floor(struct_radius/2)) 
    # and with a radius of struct_radius/2
    for i in range(0, struct_size):
        for j in range(0, struct_size):
            if((i-np.floor(struct_size/2))**2 + (j-np.floor(struct_size/2))**2 <= (struct_size/2)**2):
                structuring_element[i, j] = 1
    
    # We now perform chained opening and closing to eliminate the noise
    im = skimage.morphology.binary_opening(im, structuring_element)
    im = skimage.morphology.binary_closing(im, structuring_element)

    return im
    ### END YOUR CODE HERE ###

In [10]:
def distance_transform(im: np.ndarray) -> np.ndarray:
    """
        A function that computes the distance to the closest boundary pixel.

        args:
            im: np.ndarray of shape (H, W) with boolean values (dtype=np.bool)
        return:
            (np.ndarray) of shape (H, W). dtype=np.int32
    """
    ### START YOUR CODE HERE ### (You can change anything inside this block)
    assert im.dtype == np.bool
    structuring_element = np.array([
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]
    ], dtype=bool)
    # We start with an empty image
    result = im.astype(np.int32)
    # We perform a loop on all the possible intensity values for 8-bit images
    for i in range(0, 255):
        eroded_im = skimage.morphology.binary_erosion(im, structuring_element)
        # We get the pixels that changed (XOR) in this erosion stage and set their value to i, following
        # the chessboard pattern
        diff_im = np.logical_xor(im, eroded_im)
        result[diff_im] = i
        im = eroded_im
        # If no pixels changed, that means the image is already all 0s and we can stop
        if(np.count_nonzero(diff_im) == 0):
            break

    return result
    ### END YOUR CODE HERE ###

In [11]:
if __name__ == "__main__":
    im = utils.read_image("noisy.png")
    binary_image = (im != 0)
    noise_free_image = remove_noise(binary_image)
    distance = distance_transform(noise_free_image)

    assert im.shape == distance.shape, "Expected image shape ({}) to be same as resulting image shape ({})".format(
            im.shape, distance.shape)
    assert distance.dtype == np.int32, "Expected resulting image dtype to be np.int32. Was: {}".format(
            distance.dtype)

    distance = utils.to_uint8(distance)
    utils.save_im("noisy-distance.png", distance)

Reading image: images\noisy.png
Saving image to: image_processed\noisy-distance.png


C:\Users\Pablo\AppData\Local\Temp\ipykernel_35396\3370909493.py:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert im.dtype == np.bool
